<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%EC%84%B1%ED%95%98%EA%B8%B0(%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### 클래스 정의하기

신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화

nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_Stack = nn.Sequential(
        nn.Linear(28*28 , 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_Stack(x)
    return logits    

In [8]:
model = NeuralNetwork().to(device)   # NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤, structure를 출력
print(model)   

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_Stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1,28,28, device =device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

### 모델 계층 (Layer)

In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
- nn.Flatten : 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear
- 선형계층은 저장된 가중치와 편향을 사용하여, 입력에 선형 변환을 적용하는 모듈

In [12]:
layer1 = nn.Linear(in_features =28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
- 비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4192,  0.0787,  0.3141,  0.1675,  0.0449, -0.3343, -0.5247, -0.0913,
         -0.4430, -0.0020,  0.0027, -0.2033,  0.2395, -0.2058, -0.5908, -0.5233,
         -0.2646, -0.2044, -0.9807,  0.0985],
        [-0.5398,  0.3363,  0.2980,  0.2563,  0.0646, -0.0260, -0.4060,  0.3462,
         -0.4480, -0.0831, -0.0854, -0.3456,  0.6677, -0.1571, -0.0969, -0.4367,
         -0.2583,  0.1933, -1.0184,  0.0256],
        [-0.4910, -0.3392,  0.4846,  0.2154,  0.2794, -0.2808, -0.4195,  0.1362,
         -0.6011, -0.0779, -0.0308, -0.1356,  0.2796, -0.2558,  0.0014, -0.3653,
         -0.6978, -0.2330, -1.2220, -0.8632]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0787, 0.3141, 0.1675, 0.0449, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0027, 0.0000, 0.2395, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0985],
        [0.0000, 0.3363, 0.2980, 0.2563, 0.0646, 0.0000, 0.0000, 0.3462, 0.0000,
         0.0000, 0.0000, 0.0000, 0.6677, 0.0000, 0.00

#### nn.Sequential
- nn.Sequential : 순서를 갖는 모듈의 컨테이너

In [14]:
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20,10))
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn,Softmax
- 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 [-infty, infty] 범위의 raw 값인 logits을 반환한다. logits는 모델의 각 class에대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정(scale)


In [15]:
softmax = nn.Softmax(dim=1)   # dim=1 : 값의 합이 1이 되는 차원
pred_probab = softmax(logits)